<a href="https://colab.research.google.com/github/subbu27iitb/SVM/blob/main/SVM_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

**Telecom Chrun dataset**

In [ ]:
from google.colab import files
dataset = files.upload()

Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn.csv


In [ ]:
churn_df=pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(churn_df.shape)
churn_df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


**Transform Multiple Categotical features to Numerical features in the dataframe**

Update: "categorical_columns" vector

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder

categorical_columns = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "Churn" ]
column_trans = make_column_transformer((OrdinalEncoder(), categorical_columns))
churn_trans = column_trans.fit_transform(churn_df)
print(type(churn_trans))
churn_trans

<class 'numpy.ndarray'>


array([[0., 1., 0., ..., 1., 2., 0.],
       [1., 0., 0., ..., 0., 3., 0.],
       [1., 0., 0., ..., 1., 3., 1.],
       ...,
       [0., 1., 1., ..., 1., 2., 0.],
       [1., 1., 0., ..., 1., 3., 1.],
       [1., 0., 0., ..., 1., 0., 0.]])

**Update Encoded Categorical Features into Original Dataset**

In [ ]:
#churn_df_trans = churn_df.copy()
churn_df_trans = pd.DataFrame(churn_trans, columns=categorical_columns)
print(churn_df_trans.head())
churn_df.update(churn_df_trans)
churn_df.head()

   gender  Partner  Dependents  ...  PaperlessBilling  PaymentMethod  Churn
0     0.0      1.0         0.0  ...               1.0            2.0    0.0
1     1.0      0.0         0.0  ...               0.0            3.0    0.0
2     1.0      0.0         0.0  ...               1.0            3.0    1.0
3     1.0      0.0         0.0  ...               0.0            0.0    0.0
4     0.0      0.0         0.0  ...               1.0            2.0    1.0

[5 rows x 16 columns]


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1


**Check if there are any missing/null values**

In [ ]:
churn_df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

**Ensure all Features are numerical type not object type**

In [ ]:
print(churn_df.dtypes)

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


**Change the type of features to numeric**

Note:

i. Ensure all features are numeric type but not object.

ii. Imputation to be applied to that numeric vector.

In [ ]:
col=list(churn_df.columns)
#type(col)
churn_df.loc[:,col] = pd.to_numeric(churn_df[col].values.ravel(),'coerce').reshape(-1,len(col)) #why errors = 'coerce' required ?

print(type(col))
print(churn_df.shape)
churn_df.dtypes

<class 'list'>
(7043, 21)


customerID          float64
gender              float64
SeniorCitizen       float64
Partner             float64
Dependents          float64
tenure              float64
PhoneService        float64
MultipleLines       float64
InternetService     float64
OnlineSecurity      float64
OnlineBackup        float64
DeviceProtection    float64
TechSupport         float64
StreamingTV         float64
StreamingMovies     float64
Contract            float64
PaperlessBilling    float64
PaymentMethod       float64
MonthlyCharges      float64
TotalCharges        float64
Churn               float64
dtype: object

**Predict the missing values**

Note: In this example, there are no missing values.

In [ ]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
churn_df.iloc[:, 19] = imp_mean.fit_transform(churn_df.iloc[:, 19].values.reshape(-1,1) ) #why reshape(-1,1)

**Prepare train and test Datasets**

In [ ]:
churn_df_X = churn_df.drop(["Churn", "customerID"], axis =1)
#print(churn_df_X.head())
churn_df_y = churn_df["Churn"]
#print(churn_df_y.head())

X_train, X_test, y_train, y_test = train_test_split(churn_df_X, churn_df_y, train_size=0.75, test_size=0.25, random_state=4)
#print(X_test.head())

In [ ]:
from sklearn.svm import SVC

**Kernel = Sigmoid**

In [ ]:
SVModel = SVC(kernel = 'sigmoid', C=1, gamma = 'auto', probability = True)
SVModel.fit(X_train, y_train)

SVC(C=1, gamma='auto', kernel='sigmoid', probability=True)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
confusion_matrix(y_train, SVModel.predict(X_train))

array([[3877,    0],
       [1405,    0]])

In [ ]:
print(classification_report(y_train, SVModel.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.73      1.00      0.85      3877
         1.0       0.00      0.00      0.00      1405

    accuracy                           0.73      5282
   macro avg       0.37      0.50      0.42      5282
weighted avg       0.54      0.73      0.62      5282



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Kernel = rbf**

In [ ]:
SVModel = SVC(kernel = 'rbf', C=1, gamma = 'auto', probability = True)
SVModel.fit(X_train, y_train)
confusion_matrix(y_train, SVModel.predict(X_train))

array([[3821,   56],
       [ 144, 1261]])

In [ ]:
print(accuracy_score(y_train, SVModel.predict(X_train)))
print(classification_report(y_train, SVModel.predict(X_train)))

0.9621355547141235
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97      3877
         1.0       0.96      0.90      0.93      1405

    accuracy                           0.96      5282
   macro avg       0.96      0.94      0.95      5282
weighted avg       0.96      0.96      0.96      5282



**GRIDSEARCH CV**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param = {'kernel': ['linear', 'poly', 'sigmoid', 'rbf'],
         'C':      np.arange(1,42,10),
         'coef0':  np.arange(0.001,3, 0.5),
         'degree': np.arange(3,6),
         'gamma':  ('auto', 'scale')
    }

param

{'C': array([ 1, 11, 21, 31, 41]),
 'coef0': array([1.000e-03, 5.010e-01, 1.001e+00, 1.501e+00, 2.001e+00, 2.501e+00]),
 'degree': array([3, 4, 5]),
 'gamma': ('auto', 'scale'),
 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']}

In [ ]:
SVModel_GS = SVC()
GridS = GridSearchCV(SVModel_GS, param, cv=5)
GridS.fit(X_train, y_train)